In [ ]:
import pandas as pd
import numpy as np
import altair as alt

In [ ]:
!wget https://zenodo.org/record/583613/files/sample1-f.fq.gz
!gunzip sample1-f.fq.gz

In [ ]:
# Open the FASTQ file and read quality values
line_lengths = []
with open("sample1-f.fq", "r") as fastq_file:
    for line_number, line in enumerate(fastq_file):
        if line_number % 4 == 3:
            line = line.rstrip()
            line_lengths.append(len(line))                

In [ ]:
lengths = pd.DataFrame(np.histogram(line_lengths,bins=range(25,400,25))).T.fillna(0)

In [ ]:
lengths

In [ ]:
matrix = np.zeros((len(line_lengths),np.max(line_lengths)))

In [ ]:
matrix.shape

In [ ]:
with open("sample1-f.fq", "r") as fastq_file:
    i = 0
    for line_number, line in enumerate(fastq_file):
        if line_number % 4 == 3:
            line = line.rstrip()
            for j,char in enumerate(line):
                matrix[i][j] = ord(char)
            i += 1

In [ ]:
stats = pd.DataFrame(matrix).describe().T.reset_index()

In [ ]:
stats

In [ ]:
base = alt.Chart(stats,title='Quality score distributioin').mark_rule(color='orange',opacity=.75,strokeWidth=3).encode(
    x = alt.X('index:Q',title='Position in read'),
    y = alt.Y('25%:Q',stack='center',title='Quality score'),
    y2 = alt.Y2('75%:Q')
).properties(
    width=800,
    height=200)

median = base.mark_circle(size=10,color='red').encode(
    y = '50%:Q',   
)

len_dist = alt.Chart(lengths,title='Length distribution').mark_bar().encode(
    x = alt.X("1:N",title='Read length'),
    y = alt.Y('0:Q',title='Read count')
).properties(
    width=200,
    height=200)

(base + median) | len_dist